# CCO '07 P2 - Snowflakes
-----
**Canadian Computing Competition: 2007 Stage 2, Day 1, Problem 2**

You may have heard that no two snowflakes are alike. Your task is to write a program to determine whether this is really true. Your program will read information about a collection of snowflakes, and search for a pair that may be identical.

Each snowflake has six arms. For each snowflake, your program will be provided with a measurement of the length of each of the six arms. Any pair of snowflakes which have the same lengths of corresponding arms should be flagged by your program as possibly identical.

**Note: The original CCO data were weak and have been augmented with some custom test cases.**
## Input Specification
----
The first line of input will contain a single integer n, 0 < n ≤ 100 000 0 < n \le 100\,000 , the number of snowflakes to follow. This will be followed by n n lines, each describing a snowflake. Each snowflake will be described by a line containing six integers (each integer is at least 0 0 and less than 10 000 000 10\,000\,000 ), the lengths of the arms of the snowflake. The lengths of the arms will be given in order around the snowflake (either clockwise or counterclockwise), but they may begin with any of the six arms. For example, the same snowflake could be described as `1 2 3 4 5 6` or `4 3 2 1 6 5`.
## Output Specification
----
If all of the snowflakes are distinct, your program should print the message: `No two snowflakes are alike.`

If there is a pair of possibly identical snowflakes, your program should print the message: `Twin snowflakes found.`

## Sample Input
----
```
2
1 2 3 4 5 6
4 3 2 1 6 5
```
## Output for Sample Input
----
```
Twin snowflakes found.
```

The Linux-user.gr submission below uses a range of 0-255 as snowflake arm length.

## Let's Create a dataset

Also let's make a function for loading the data from the file

In [30]:
from datetime import datetime
import random
import os
import math
import collections

NUMBER_OF_SNOWFLAKES = 10000
MAX_SNOWFLAKE_LENGTH = 255
NUMBER_OF_SNOWFLAKE_BRANCHES = 6
# First create a dataset
def create_dataset():
    start = datetime.now()
    f = open("dataset.txt", "w")
    f.write(str(NUMBER_OF_SNOWFLAKES) + "\n")
    for x in range(0, NUMBER_OF_SNOWFLAKES):
        for y in range(0, 5):
            f.write(str(random.randint(0, MAX_SNOWFLAKE_LENGTH)) + " ")
        f.write(str(random.randint(0, MAX_SNOWFLAKE_LENGTH)) + "\n")
    f.close()
    finish = datetime.now()
    print("Dataset created in {}".format((finish - start)))

create_dataset()

Dataset created in 0:00:00.072370


In [31]:
def read_dataset():
    ds = []
    start = datetime.now()
    f = open("dataset.txt", "r")
    n = f.readline()
    for i in range(int(n)):
        line = f.readline()
        line_data = line.strip('\n').split(' ')
        data_points = []
        for point in line_data:
            data_points.append(int(point))
        ds.append(data_points)
    f.close()
    finish = datetime.now()
    print("Dataset loaded in {}".format((finish - start)))
    return ds

## Solution
----

A simple solution is to just compare all snowflakes making all rotations.

In [32]:
def basic_comparator(snowflake, next_snowflake):
    alike_flag = False
    for x in range(len(snowflake)):
        for y in range(len(next_snowflake)):
            if snowflake[x] == next_snowflake[y]:
                for z in range(1, 5):
                    next_x = x + z
                    if next_x >= NUMBER_OF_SNOWFLAKE_BRANCHES:
                        next_x = next_x - NUMBER_OF_SNOWFLAKE_BRANCHES
                    next_y = y + z
                    if next_y >= NUMBER_OF_SNOWFLAKE_BRANCHES:
                        next_y = next_y - NUMBER_OF_SNOWFLAKE_BRANCHES
                    if snowflake[next_x] != next_snowflake[next_y]:
                        alike_flag = False
                        break
                    else:
                        alike_flag = True
    rev_flag = False
    rev_snow = list(reversed(next_snowflake))
    for x in range(len(snowflake)):
        for y in range(len(rev_snow)):
            if snowflake[x] == rev_snow[y]:
                for z in range(1, 5):
                    next_x = x + z
                    if next_x >= NUMBER_OF_SNOWFLAKE_BRANCHES:
                        next_x = next_x - NUMBER_OF_SNOWFLAKE_BRANCHES
                    next_y = y + z
                    if next_y >= NUMBER_OF_SNOWFLAKE_BRANCHES:
                        next_y = next_y - NUMBER_OF_SNOWFLAKE_BRANCHES
                    if snowflake[next_x] != rev_snow[next_y]:
                        rev_flag = False
                        break
                    else:
                        rev_flag = True
    if alike_flag or rev_flag:
        return True
    else:
        return False


def find_duplicate_snowflakes(comparator, tick):
    # Read dataset
    snowflakes = read_dataset()
    flag = False
    # Compare snowflakes
    for i in range(len(snowflakes)-1):
        if i % (NUMBER_OF_SNOWFLAKES / 10) == 0 and i !=0:
            tock = datetime.now()
            print("processed {}/{} in {}".format(i, NUMBER_OF_SNOWFLAKES, tock - tick))
        for j in range(i+1, len(snowflakes)):
            temp_flag = comparator(snowflakes[i], snowflakes[j])
            if temp_flag:
                flag = True
    if flag:
        print("Twin snowflakes found.")
    else:
        print("No two snowflakes are alike.")
    return flag

In [33]:
print('LinuxUser Programming Exercise No 14')
print('Basic Solution: all rotations')
tick = datetime.now()
find_duplicate_snowflakes(basic_comparator, tick)
tock = datetime.now()
print("Basic Solution finished in {}".format((tock - tick)))

LinuxUser Programming Exercise No 14
Basic Solution: all rotations
Dataset loaded in 0:00:00.014185
processed 1000/10000 in 0:00:53.135646
processed 2000/10000 in 0:01:41.180041
processed 3000/10000 in 0:02:24.152262
processed 4000/10000 in 0:03:01.145790
processed 5000/10000 in 0:03:30.965116
processed 6000/10000 in 0:03:55.253865
processed 7000/10000 in 0:04:14.103377
processed 8000/10000 in 0:04:27.642320
processed 9000/10000 in 0:04:35.776307
No two snowflakes are alike.
Basic Solution finished in 0:04:38.838164


## A Better Comparison
The simple solution may work, but it is too slow. In order to make it faster we can use a number of different tricks. One of them is to reverse the question. What if instead of searching for twin snowflakes we immediately stop processing two snowflakes when they are not twins. One easy way to do that is by checking if they contain the same elements (without caring about the order).

In [34]:
def better_comparator(snowflake, next_snowflake):
    alike_flag = False
    if collections.Counter(snowflake) != collections.Counter(next_snowflake):
        return alike_flag
    alike_flag = basic_comparator(snowflake, next_snowflake)
    return alike_flag

In [35]:
print('Better Comparator Solution: Do rotations only when snowflakes contain the same elements  (reversed criteria)')
tick = datetime.now()
find_duplicate_snowflakes(better_comparator, tick)
tock = datetime.now()
print("Better Solution finished in {}".format((tock - tick)))

Better Comparator Solution: Do rotations only when snowflakes contain the same elements  (reversed criteria)
Dataset loaded in 0:00:00.016167
processed 1000/10000 in 0:00:24.227850
processed 2000/10000 in 0:00:46.590173
processed 3000/10000 in 0:01:06.291286
processed 4000/10000 in 0:01:23.389071
processed 5000/10000 in 0:01:37.746473
processed 6000/10000 in 0:01:49.246863
processed 7000/10000 in 0:01:58.657306
processed 8000/10000 in 0:02:05.114270
processed 9000/10000 in 0:02:08.721853
No two snowflakes are alike.
Better Solution finished in 0:02:09.989461


An even faster comparison would be to use the sum of arm lengths of each snowflake. Comparing one number is faster than comparing the whole set of elements. If 2 sets have the same sum, we then compare the set and if the set has the same numbers we then do all the rotations for given snowflakes.

In [36]:
def sum_comparator(snowflake, next_snowflake):
    alike_flag = False
    if sum(snowflake) != sum(next_snowflake):
        return alike_flag
    if collections.Counter(snowflake) != collections.Counter(next_snowflake):
        return alike_flag
    alike_flag = basic_comparator(snowflake, next_snowflake)
    return alike_flag

In [37]:
print('Sum Comparator Solution: Do rotations only when sum of arm lengths is equal (reversed criteria)')
tick = datetime.now()
find_duplicate_snowflakes(sum_comparator, tick)
tock = datetime.now()
print("Sum Comparator finished in {}".format((tock - tick)))

Sum Comparator Solution: Do rotations only when sum of arm lengths is equal (reversed criteria)
Dataset loaded in 0:00:00.018275
processed 1000/10000 in 0:00:03.102474
processed 2000/10000 in 0:00:05.553560
processed 3000/10000 in 0:00:08.687856
processed 4000/10000 in 0:00:10.734337
processed 5000/10000 in 0:00:12.289837
processed 6000/10000 in 0:00:13.566446
processed 7000/10000 in 0:00:14.644220
processed 8000/10000 in 0:00:15.583376
processed 9000/10000 in 0:00:16.146605
No two snowflakes are alike.
Sum Comparator finished in 0:00:16.318444


## A Better Algorithm
The Comparator function can be further improved, but we will keep it for now and try to improve on the algorithm. The current algorithm starts by choosing the first snowflake and comparing it with all the rest. Then it chooses the second snowflake and compares it with all the other snowflakes except the first, and so on.

One simple improvement would be to stop comparing the moment we find at least a single pair of twin snowflakes. This will improve our best case scenario, but overall worst case time will remain the same.

In [38]:
def better_find_duplicate_snowflakes(comparator, tick):
    # Read dataset
    snowflakes = read_dataset()
    flag = False
    # Compare snowflakes
    for i in range(len(snowflakes)-1):
        if i % (NUMBER_OF_SNOWFLAKES / 10) == 0 and i !=0:
            tock = datetime.now()
            print("processed {}/{} in {}".format(i, NUMBER_OF_SNOWFLAKES, tock - tick))
        for j in range(i+1, len(snowflakes)):
            flag = comparator(snowflakes[i], snowflakes[j])
            if flag:
                print("Twin snowflakes found.")
                return flag
    print("No two snowflakes are alike.")
    return flag

In [39]:
print('Basic Solution: all rotations, stop on first twins found')
tick = datetime.now()
better_find_duplicate_snowflakes(basic_comparator, tick)
tock = datetime.now()
print("Basic Solution finished in {}".format((tock - tick)))

Basic Solution: all rotations, better algorithm
Dataset loaded in 0:00:00.015044
processed 1000/10000 in 0:00:50.520034
processed 2000/10000 in 0:01:37.682892
processed 3000/10000 in 0:02:17.083803
processed 4000/10000 in 0:02:50.990768
processed 5000/10000 in 0:03:19.727623
processed 6000/10000 in 0:03:43.301993
processed 7000/10000 in 0:04:01.264001
processed 8000/10000 in 0:04:14.361530
processed 9000/10000 in 0:04:22.099216
No two snowflakes are alike.
Basic Solution finished in 0:04:25.042713


Another, idea would be to use a Sum Dictionary. Instead of storing our snowflakes on a list, we will create a dictionary of lists. Each list will contain all snowflakes with the same sum of arm lengths. The sums will represent the keys of the dictionary. When our dictionary is complete, we will run our ```better_comparator``` only for the snowflakes in groups with equal sum. Instead of looping through the whole dataset we loop only through small groups.

In [40]:
def sum_dictionary_find_duplicate_snowflakes(comparator, tick):
    # Read dataset
    snowflakes = read_dataset()
    dict_snowflakes = dict()
    for snowflake in snowflakes:
        temp_sum = sum(snowflake)
        if temp_sum not in dict_snowflakes:
            dict_snowflakes[temp_sum] = [snowflake]
        elif isinstance(dict_snowflakes[temp_sum], list):
            dict_snowflakes[temp_sum].append(snowflake)
        else:
            dict_snowflakes[temp_sum] = [snowflake]
    flag = False
    z = 0
    for list_snow in dict_snowflakes.values():
        if len(list_snow) > 1:
            if z % 100 == 0 and z !=0:
                tock = datetime.now()
                print("processed {} group in {}".format(z, tock - tick))
            # Compare snowflakes
            for i in range(len(list_snow)-1):
                for j in range(i+1, len(list_snow)):
                    flag = comparator(list_snow[i], list_snow[j])
                    if flag:
                        print("Twin snowflakes found.")
                        return flag
            z+=1
    print("No two snowflakes are alike.")
    return flag

In [41]:
print('Sum Dictionary Solution: Comparison only for snowflakes with same sum')
tick = datetime.now()
sum_dictionary_find_duplicate_snowflakes(better_comparator, tick)
tock = datetime.now()
print("Sum Dictionary Solution finished in {}".format((tock - tick)))

Sum Dictionary Solution: Comparison only for snowflakes with same sum
Dataset loaded in 0:00:00.015807
processed 100 group in 0:00:00.057089
processed 200 group in 0:00:00.094209
processed 300 group in 0:00:00.126221
processed 400 group in 0:00:00.154346
processed 500 group in 0:00:00.181947
processed 600 group in 0:00:00.200030
processed 700 group in 0:00:00.214456
processed 800 group in 0:00:00.217851
No two snowflakes are alike.
Sum Dictionary Solution finished in 0:00:00.218920


## Results
Finally, let's see the results of different algorithms, for 10.000 snowflakes with arm lengths defined in [0,255] and 6 arms per snowflake.

| Algorithm                                                                                                    | Time           |
|--------------------------------------------------------------------------------------------------------------|----------------|
| Basic Solution: All rotations                                                                                | 0:04:38.838164 |
| Better Comparator Solution: Do rotations only when snowflakes contain the same elements  (reversed criteria) | 0:02:09.989461 |
| Sum Comparator Solution: Do rotations only when sum of arm lengths is equal (reversed criteria)              | 0:00:16.318444 |
| Basic Solution: all rotations, stop on first twins found                                                     | 0:04:25.042713 |
| Sum Dictionary Solution: Comparison only for snowflakes with same sum                                        | 0:00:00.218920 |